In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
data=pd.read_csv('../input/tabular-playground-series-apr-2021/train.csv')

In [ ]:
cols=['Sex','Ticket','Cabin','Embarked']

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder() 
for i in cols:
    data[i]= label_encoder.fit_transform(data[i])

In [ ]:
data.isnull().sum()

In [ ]:
data=data.fillna(data.mean())

In [ ]:
data.describe()

In [ ]:
len(data[data['Age']==0])

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
col=['Pclass', 'Sex', 'Age', 'SibSp','Parch','Ticket', 'Fare', 'Cabin', 'Embarked']

In [ ]:
X=data[col]
Y=data['Survived']

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

In [ ]:
lsvc = LinearSVC(C=0.001, penalty="l1", dual=False).fit(X, Y)

In [ ]:
model2 = SelectFromModel(lsvc, prefit=True)
X_new = model2.transform(X)

In [ ]:
X_new.shape

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X_new,Y,test_size=0.15)

In [ ]:
import keras 
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

In [ ]:
def create_model():
        model = Sequential()
        model.add(Dense(100,kernel_initializer='uniform',input_dim=8,activation='relu'))
        model.add(Dense(400,kernel_initializer='uniform',activation='relu'))
        model.add(Dense(400,kernel_initializer='uniform',activation='relu'))
        model.add(Dense(400,kernel_initializer='uniform',activation='relu'))
        model.add(Dense(400,kernel_initializer='uniform',activation='relu'))
        model.add(Dense(400,kernel_initializer='uniform',activation='relu'))
        model.add(Dense(400,kernel_initializer='uniform',activation='relu'))
        model.add(Dense(1,kernel_initializer='uniform', activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

In [ ]:
model=create_model()
model.summary()

In [ ]:
hist=model.fit(np.asarray(train_x).astype('float32'),train_y,epochs=100,batch_size=128)

In [ ]:
model.evaluate(test_x,test_y)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
classifier = KerasClassifier(build_fn = create_model)
param_grid = dict(epochs=[50,100,200],
                  batch_size=[16,32,64,128])
grid = HalvingRandomSearchCV(classifier, param_grid, scoring='accuracy',cv=10,n_jobs=-1)
grid_result = grid.fit(train_x, train_y)
best_parameters = grid.best_params_
best_accuracy = grid.best_score_

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
classifier = KerasClassifier(build_fn = create_model,
                             batch_size=best_parameters['batch_size'],
                             epochs=best_parameters['epochs'])

classifier.fit(train_x,train_y)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
oversample = RandomOverSampler(sampling_strategy=1)

In [ ]:
X, Y = oversample.fit_resample(X, Y)

In [ ]:
len(X)

In [ ]:
test=pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')
test2=pd.read_csv('../input/tabular-playground-series-apr-2021/test.csv')

In [ ]:
for i in cols:
    test[i]= label_encoder.fit_transform(test[i])

In [ ]:
test=test.fillna(test.mean())

In [ ]:
X_t=test[col]

In [ ]:
X_new_t = model2.transform(X_t)

In [ ]:
y_pred=model.predict(np.asarray(X_new_t).astype('float32'))

In [ ]:
def app(y_pred):
    pred=[]
    for i in y_pred:
        
        if i[0]>0.500000001:
            pred.append(1)
        else:
            pred.append(0)
    return pred

In [ ]:
pred=app(y_pred)

In [ ]:
submission = pd.DataFrame({'PassengerId':test2.PassengerId,'Survived':pred})
submission.to_csv('subm.csv',index=False)